In [1]:
!pip install peft
!pip install sacrebleu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 5.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 3.5 MB/s eta 0:00:00


In [2]:
import torch
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer
from peft import LoraConfig, get_peft_model, PeftModel, prepare_model_for_kbit_training
from datasets import load_dataset
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

In [3]:
model_name = "facebook/m2m100_1.2B"
tokenizer = M2M100Tokenizer.from_pretrained(model_name)
model = M2M100ForConditionalGeneration.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/909 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

In [4]:
special_tokens = {"additional_special_tokens": ["__mns__"]}
tokenizer.add_special_tokens(special_tokens)
model.resize_token_embeddings(len(tokenizer))

M2M100ScaledWordEmbedding(128105, 1024, padding_idx=1)

In [5]:
lora_config = LoraConfig(
    r=16,  
    lora_alpha=32,  
    target_modules=["q_proj", "v_proj"],  
    lora_dropout=0.1,  
    bias="none", 
    task_type="SEQ_2_SEQ_LM"  
)

In [6]:
model = get_peft_model(model, lora_config)

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset
import sacrebleu

In [8]:
df = pd.read_csv("/kaggle/input/mansi-russian-parralel-corpus/overall_80K.csv", index_col=0)
train_df, test_val_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(test_val_df, test_size=0.5, random_state=42)

In [9]:
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

In [10]:
def preprocess_function(examples):
    targets = [f"__mns__ {inp}" for inp in examples['target']]
    inputs = [f"__ru__ {inp}" for inp in examples['source']]
    
    inputs = tokenizer(inputs, truncation=True, padding='max_length', max_length=128)
    targets = tokenizer(targets, truncation=True, padding='max_length', max_length=128)
    inputs['labels'] = targets['input_ids']
    return inputs

In [11]:
train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)

train_dataset = train_dataset.remove_columns(['source', 'target', '__index_level_0__'])
val_dataset = val_dataset.remove_columns(['source', 'target', '__index_level_0__'])
test_dataset = test_dataset.remove_columns(['source', 'target', '__index_level_0__'])

Map:   0%|          | 0/64916 [00:00<?, ? examples/s]

Map:   0%|          | 0/8115 [00:00<?, ? examples/s]

Map:   0%|          | 0/8115 [00:00<?, ? examples/s]

In [12]:
import torch
torch.cuda.empty_cache()

In [13]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./m2m-1B-finetune",
    evaluation_strategy="no",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4, 
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="no", 
    predict_with_generate=True,
    fp16=True,  
    logging_dir="./logs",
    report_to="none", 
    load_best_model_at_end=False, 
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer
)

trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Step,Training Loss
500,7.674300
1000,6.553800
1500,6.454200
2000,6.398000
2500,6.377500
3000,6.341000
3500,6.320600
4000,6.301800
4500,6.283600
5000,6.268500


TrainOutput(global_step=12171, training_loss=6.331619394333868, metrics={'train_runtime': 25058.3739, 'train_samples_per_second': 7.772, 'train_steps_per_second': 0.486, 'total_flos': 1.6645751374558003e+17, 'train_loss': 6.331619394333868, 'epoch': 2.999815145726785})

In [14]:
model.eval();
!mkdir m2m_1B_finetune
!mkdir m2m_1B_finetune/model
!mkdir m2m_1B_finetune/tokenizer
model.save_pretrained("m2m_1B_finetune/model/")
tokenizer.save_pretrained("m2m_1B_finetune/tokenizer")

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:232: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


('m2m_1B_finetune/tokenizer/tokenizer_config.json',
 'm2m_1B_finetune/tokenizer/special_tokens_map.json',
 'm2m_1B_finetune/tokenizer/vocab.json',
 'm2m_1B_finetune/tokenizer/sentencepiece.bpe.model',
 'm2m_1B_finetune/tokenizer/added_tokens.json')

In [15]:
import shutil
from IPython.display import FileLink

shutil.make_archive("m2m_1B_finetune", 'zip', "m2m_1B_finetune/")
FileLink('m2m_1B_finetune.zip')

/kaggle/working/m2m_1B_finetune.zip

In [ ]:
import random
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

random.seed(42) 
random_test_samples = test_dataset
model.eval()

def generate_translation(sample):
    input_ids = torch.tensor(sample['input_ids']).to(device)  # Конвертируем список в тензор
    with torch.no_grad():
        generated_ids = model.generate(input_ids)
    return tokenizer.decode(generated_ids[0], skip_special_tokens=True)

original_texts = []
correct_translations = []
model_translations = []

for sample in tqdm(random_test_samples):
    input_text = tokenizer.decode(sample['input_ids'], skip_special_tokens=True)
    correct_translation = tokenizer.decode(sample['labels'], skip_special_tokens=True)
    model_translation = generate_translation(sample)

    original_texts.append(input_text)
    correct_translations.append(correct_translation)
    model_translations.append(model_translation)
    
df_results = pd.DataFrame({
    "Original Text": original_texts,
    "Correct Translation": correct_translations,
    "Model Translation": model_translations
})

In [ ]:
df_results.head(50)

In [ ]:
df_results.loc[:, 'Original Text'] = df_results['Original Text'].str.replace('__en__', '')
df_results.loc[:, 'Correct Translation'] = df_results['Correct Translation'].str.replace('__en__', '')
df_results.loc[:, 'Model Translation'] = df_results['Model Translation'].str.replace('__en__', '')
    
bleu_score = sacrebleu.corpus_bleu(df_results['Model Translation'].tolist(), 
                                   [df_results['Correct Translation'].tolist()]).score
chrf_score = sacrebleu.corpus_chrf(df_results['Model Translation'].tolist(), 
                                   [df_results['Correct Translation'].tolist()]).score

print(f"BLEU Score: {bleu_score}")
print(f"ChrF Score: {chrf_score}")

df_results.head(50)